In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import pandas as pd

In [3]:
conf = SparkConf().setAppName("sendToMysql").setMaster("spark://master:7077").set("spark.jars", "/opt/spark/jars/mysql-connector-java-8.0.26.jar")

In [4]:
sc = SparkContext(conf=conf)

2021-08-20 15:34:03,508 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
sqlCtx = SQLContext(sc)
spark = sqlCtx.sparkSession

In [6]:
import subprocess
import re
cmd = '/opt/hadoop/bin/hdfs dfs -ls /streaming/out'
files = subprocess.check_output(cmd, shell=True).decode('utf8').strip().split('\n')

file_list=[]
for path in files:
    file = re.search(r'/streaming/out/.*json',path)
    if file != None:
        file_list.append(file.group())

print(file_list)

['/streaming/out/part-00000-116fade8-5820-4c8a-9433-c148d89c1c6e-c000.json', '/streaming/out/part-00000-26e14bd2-a4bf-4e39-a270-c909361472a8-c000.json', '/streaming/out/part-00000-6bca0547-c676-42cb-86a5-1038567fb9e1-c000.json', '/streaming/out/part-00000-6cd65886-0337-4050-aa7f-1b4499e4f82e-c000.json', '/streaming/out/part-00000-76eddc06-25d7-4242-955a-cdae349f6d74-c000.json', '/streaming/out/part-00000-bbb06dd6-2ca6-4c82-bdf5-2feb0d018de9-c000.json', '/streaming/out/part-00001-0be8d1f7-dad0-493e-84f0-5bda6aa7f66d-c000.json', '/streaming/out/part-00001-4164ee14-41fe-4586-ba76-4314764c18bd-c000.json', '/streaming/out/part-00001-4ee2f95d-90ee-4b9d-86a3-d5975a7c062b-c000.json', '/streaming/out/part-00001-b331a820-d269-41dd-ac0a-463dd9e5da96-c000.json', '/streaming/out/part-00001-bd51e116-8da2-4d68-8e59-7b4fd3ee0757-c000.json', '/streaming/out/part-00002-155ed2d8-3a80-4854-bee5-848d9ec2ed05-c000.json', '/streaming/out/part-00002-2f166f28-e69a-4833-8400-dd63318e66d3-c000.json', '/streaming

In [7]:
df_load = spark.read.json('hdfs://'+file_list[0])
df_load.show()

+--------------------+
|               value|
+--------------------+
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
|{"year": "2020", ...|
+--------------------+
only showing top 20 rows



In [8]:
df = df_load.select("*").toPandas()

In [9]:
import json
df_list = []
for value in df['value']:
    df_list.append(json.loads(value))

In [10]:
df_list

[{'year': '2020',
  'month': '11',
  'day': '25',
  'r_hour': '5',
  'avg(value)': '10.4',
  'real_value': '10.9'},
 {'year': '2020',
  'month': '11',
  'day': '25',
  'r_hour': '12',
  'avg(value)': '6.5',
  'real_value': '4.3'},
 {'year': '2020',
  'month': '11',
  'day': '25',
  'r_hour': '20',
  'avg(value)': '3.2',
  'real_value': '3.5'},
 {'year': '2020',
  'month': '11',
  'day': '25',
  'r_hour': '0',
  'avg(value)': '4',
  'real_value': '7.3'},
 {'year': '2020',
  'month': '11',
  'day': '26',
  'r_hour': '6',
  'avg(value)': '11',
  'real_value': '11.5'},
 {'year': '2020',
  'month': '11',
  'day': '26',
  'r_hour': '7',
  'avg(value)': '10.75',
  'real_value': '10.3'},
 {'year': '2020',
  'month': '11',
  'day': '26',
  'r_hour': '20',
  'avg(value)': '5',
  'real_value': '6.1'},
 {'year': '2020',
  'month': '11',
  'day': '27',
  'r_hour': '12',
  'avg(value)': '4.666666667',
  'real_value': '4.1'},
 {'year': '2020',
  'month': '11',
  'day': '27',
  'r_hour': '17',
  'avg(

In [11]:
temp = pd.DataFrame(df_list)

In [12]:
temp.rename(columns={'avg(value)':'avg_value'},inplace=True)

In [13]:
new_df = spark.createDataFrame(pd.DataFrame(temp))

In [15]:
sql_url = "3.34.160.1"
user = "root"
password = "12345678"
database = "multi_db"
table = "weather3"

In [16]:
new_df.write.format("jdbc")\
                .option("driver", "com.mysql.jdbc.Driver")\
                .option("url", "jdbc:mysql://{}:3306/{}?serverTimezone=Asia/Seoul ".format(sql_url, database))\
                .option("user", user)\
                .option("password", password)\
                .option("dbtable", table)\
                .mode('append').save()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
